In [93]:
import pandas as pd
import numpy as np
import datetime
import re

In [94]:
df = pd.read_csv('/Users/vadvad/Desktop/петрович/Dataset_points.csv',sep='\t',parse_dates=True)

In [95]:
# умножаем на -1 те значения очков которые списываются, чтобы при суммировании они вычлись 
df['points'] = np.where(df.recordkind == 1, -1*df.points, df.points)

In [96]:
#колонку со временем делаем дэйтайм
df.period = pd.to_datetime(df.period)
#делаем колонку с месяцем транзакции
df['month'] = df.period.dt.month
#делаем колонку с отчетным месяцем( в котором будут учитываться быллы)
df['result_month'] = 0
df['result_month'] = np.where(df.month != 12, df.month+1, 1)
#делаем колонку с годом транзакции
df['year'] = df.period.dt.year
#делаем колонку с отчетным годом( в котором будут учитываться быллы)
df['result_year'] = np.where(df.result_month != 1, df.year, df.year+1)
#приводим его в нужный формат
df['result_date'] = df.result_year.map(str)+'-'+df.result_month.map(str)+'-'+'1'
#приводим его в дэйттайм
df['result_date'] = pd.to_datetime(df.result_date)

In [97]:
#отбираем нужные столбцы
group_columns = df.groupby(['result_date','customerid']).agg({'points':'sum'}).sort_values(by='result_date').reset_index()

In [98]:
#группирем с кумулятивной суммой
cumsum_group = group_columns.groupby(by=['result_date','customerid']).sum().groupby(level=[1]).cumsum().reset_index()

In [99]:
#делаем остатки целыми числами
cumsum_group.points = cumsum_group.points.map(int)

In [100]:
#убираем нулевые остатки
cumsum_group = cumsum_group.query('points > 0')

In [102]:
cumsum_group.rename(columns={'result_date':'date'},inplace=True)

In [105]:
cumsum_group.to_csv('petro1.csv',index=False)

In [106]:
cumsum_group.to_excel('petro1.xlsx',index=False)